In [36]:
# import numpy as np
# import open3d as o3d
# import cv2
# import os

# current_dir = os.getcwd()

# def load_camera_intrinsics(stereomap_file):
#     cv_file = cv2.FileStorage(stereomap_file, cv2.FILE_STORAGE_READ)
    
#     # Get camera matrix from stereoMap.xml
#     camera_matrix = cv_file.getNode('cameraMatrixL').mat()
    
#     # Extract intrinsic parameters
#     fx = camera_matrix[0, 0]
#     fy = camera_matrix[1, 1]
#     cx = camera_matrix[0, 2]
#     cy = camera_matrix[1, 2]
    
#     cv_file.release()
#     return fx, fy, cx, cy

# def create_point_cloud(color_file, depth_file, stereomap_file):
#     # Load camera intrinsics
#     fx, fy, cx, cy = load_camera_intrinsics(stereomap_file)
    
#     # Create Open3D intrinsics
#     intrinsic = o3d.camera.PinholeCameraIntrinsic(
#         width=640, height=480,
#         fx=fx, fy=fy,
#         cx=cx, cy=cy
#     )
    
#     # Load color image
#     color_raw = o3d.io.read_image(color_file)
    
#     # Load depth data (in millimeters)
#     depth_raw = np.load(depth_file)
    
#     # Inspect depth data
#     print(f"Depth statistics:")
#     print(f"  Shape: {depth_raw.shape}")
#     print(f"  Valid points: {np.sum(depth_raw > 0)}")
#     print(f"  Min depth: {np.min(depth_raw[depth_raw > 0]):.2f} mm")
#     max_depth_mm = np.max(depth_raw[depth_raw > 0])
#     print(f"  Max depth: {max_depth_mm:.2f} mm")
#     print(f"  Mean depth: {np.mean(depth_raw[depth_raw > 0]):.2f} mm")
    
# # Handle invalid depth values
#     valid_mask = (depth_raw > 0) & np.isfinite(depth_raw)
#     if np.sum(valid_mask) == 0:
#         print("ERROR: No valid depth points found!")
#         return None
    
#     # Convert depth to uint16 format, keeping it in millimeters
#     depth_image = np.clip(depth_raw, 0, 65535).astype(np.uint16)
#     depth_o3d = o3d.geometry.Image(depth_image)    

#     # Set depth scale and truncation
#     depth_scale = 1000.0  # Convert mm to meters (depth_image / depth_scale)
#     depth_trunc = 10.0    # Maximum depth in meters (adjust as needed)

#     # Determine appropriate depth scale
#     max_depth = np.max(depth_raw[valid_mask])
#     if max_depth > 100:  
#         depth_scale = 50.0
#         depth_trunc = 520.0  
#     else:  # Likely in meters  
#         depth_scale = 1000.0
#         depth_trunc = 8.0
    
#     print(f"Using depth_scale: {depth_scale}, depth_trunc: {depth_trunc} meters")
    
#     # Create RGBD image
#     rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
#         color_raw, depth_o3d,
#         depth_scale=depth_scale,
#         depth_trunc=depth_trunc,
#         convert_rgb_to_intensity=False
#     )
    
#     # Create point cloud
#     pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
#         rgbd_image,
#         intrinsic
#     )
    
#     print(f"Point cloud created with {len(pcd.points)} points")
    
#     # Flip the orientation to match standard coordinate system
#     pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    
#     return pcd

# # def create_point_cloud(color_file, depth_file, stereomap_file):
# #     # Load camera intrinsics
# #     fx, fy, cx, cy = load_camera_intrinsics(stereomap_file)
    
# #     # Create Open3D intrinsics
# #     intrinsic = o3d.camera.PinholeCameraIntrinsic(
# #         width=640, height=480,
# #         fx=fx, fy=fy,
# #         cx=cx, cy=cy
# #     )
    
# #     # Load color image
# #     color_raw = o3d.io.read_image(color_file)
    
# #     # Load depth data and convert to Open3D format
# #     depth_raw = np.load(depth_file)  # Load the raw depth data
    
# #     # Convert depth to uint16 format required by Open3D
# #     depth_scale = 1000.0  # Scale factor to convert to millimeters
# #     depth_o3d = o3d.geometry.Image(
# #         (depth_raw * depth_scale).astype(np.uint16)
# #     )
    
# #     # Create RGBD image
# #     rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
# #         color_raw, depth_o3d,
# #         depth_scale=depth_scale,
# #         depth_trunc=5.0,  # Maximum depth in meters
# #         convert_rgb_to_intensity=False
# #     )
    
# #     # Create point cloud
# #     pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
# #         rgbd_image,
# #         intrinsic
# #     )
    
# #     # Flip the orientation to match standard coordinate system
# #     pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    
# #     return pcd

# def main():
#     current_dir = os.getcwd()
#     stereomap_file = os.path.join(current_dir, "stereoMap.xml")
    
#     # Specify your saved depth and color files
#     color_file = os.path.join(current_dir, "images", "rect_L_color.png")
#     depth_file = os.path.join(current_dir, "images", "depth_raw.npy")

#     # Create point cloud
#     pcd = create_point_cloud(color_file, depth_file, stereomap_file)
    
#     # Visualize the point cloud
#     o3d.visualization.draw_geometries([pcd])
    
#     # Optionally save the point cloud
#     o3d.io.write_point_cloud(f"pointcloud.ply", pcd)

# if __name__ == "__main__":
#     main()

In [38]:
import numpy as np
import cv2
import os
import open3d as o3d

# Load depth_raw.npy
current_dir = os.getcwd()
depth_file = os.path.join(current_dir, "images", "depth_raw.npy")
depth_map = np.load(depth_file)
print("Loaded depth_map shape:", depth_map.shape)
print("Depth map min/max values:", np.min(depth_map), np.max(depth_map))
print("Number of valid (non-zero) depth points:", np.sum(depth_map > 0))

# Load Q matrix from stereoMap.xml
cv_file = cv2.FileStorage(
    os.path.join(current_dir, "stereoMap.xml"), cv2.FILE_STORAGE_READ
)
Q = cv_file.getNode("Q").mat()
cv_file.release()
print("Loaded Q matrix:\n", Q)
print("Q matrix shape:", Q.shape)

# Create 2D grid of pixel coordinates
h, w = depth_map.shape
x, y = np.meshgrid(np.arange(w), np.arange(h))
points_3d = cv2.reprojectImageTo3D(depth_map, Q)
print("Reprojected points_3d shape:", points_3d.shape)
print("Points_3d min/max values:", np.min(points_3d), np.max(points_3d))

# Filter out invalid points (where depth is 0)
mask = depth_map > 0
points_3d = points_3d[mask]
print("Number of valid 3D points after masking:", points_3d.shape[0])

# Placeholder for colors (can be enhanced later)
colors = np.zeros_like(points_3d)

# Create Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_3d)
pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)

# Visualize the point cloud
print("Visualizing point cloud with", points_3d.shape[0], "points...")
o3d.visualization.draw_geometries([pcd], width=1280, height=720)
print("Visualization closed.")

Loaded depth_map shape: (480, 640)
Depth map min/max values: 0.0 10085442000.0
Number of valid (non-zero) depth points: 241504
Loaded Q matrix:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -4.40030807e+02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -2.49206165e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.48101560e+02]
 [ 0.00000000e+00  0.00000000e+00  2.86845756e-02  0.00000000e+00]]
Q matrix shape: (4, 4)
Reprojected points_3d shape: (480, 640, 3)
Points_3d min/max values: -inf inf
Number of valid 3D points after masking: 241504
Visualizing point cloud with 241504 points...
Visualization closed.
